In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import time
import random
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from tqdm import tqdm
import os

In [3]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [5]:
def read_data(data_type, hla_sequence):
    pos_neg_data_pd = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/' + data_type + '_pos_neg_data.csv', index_col = 0).drop_duplicates().reset_index(drop = True)
    print('# Samples = ', len(pos_neg_data_pd))
    pos_neg_data_pd = pd.merge(pos_neg_data_pd, hla_sequence, on = 'HLA')
    print('After delete unknown sequence HLA: # Samples = ', len(pos_neg_data_pd))
    return pos_neg_data_pd

In [7]:
hla_sequence = pd.read_csv('/home/chujunyi/5_ZY_MHC/data/common_hla_sequence.csv')
train_set = read_data('train', hla_sequence)
independent_set = read_data('evaluation', hla_sequence)
external_set = read_data('experimental', hla_sequence)

# Samples =  2178971
After delete unknown sequence HLA: # Samples =  2156796
# Samples =  174437
After delete unknown sequence HLA: # Samples =  172580
# Samples =  104209
After delete unknown sequence HLA: # Samples =  104209


# 去掉多余的负样本

In [8]:
def Remove_excess_negative_samples(data, train_set, type_ = 'train', savepath = True):
    idx = []
    for hla in tqdm(set(data.HLA)):
        for length in range(8,15):
            if type_ == 'train':
                for fold in [0.0, 1.0, 2.0, 3.0, 4.0]:
                    try:
                        idx.append(data[data.HLA == hla][data.length == length][data.neg_fold == fold].index[0])
                        continue
                    except:
                        continue
            else:
                try:
                    idx.append(data[data.HLA == hla][data.length == length][data.label == 0].index[0])
                    continue
                except:
                    continue
                        
    print('Remove num = ', len(idx))
    print('Original: ', Counter(data.label))
    data.drop(index = idx, axis = 0, inplace = True)
    data.reset_index(drop = True, inplace = True)
    print('After remove: ', Counter(data.label))
    
    # 去掉independent和external中在train set中的数据
    if type_ != 'train':
        copy_train_set = deepcopy(train_set)
        del copy_train_set['neg_fold']
        data = pd.concat([data, copy_train_set, copy_train_set]).drop_duplicates(['peptide', 'HLA'], keep = False).reset_index(drop = True)
        print('After remove duplicates with train_set: ', Counter(data.label))
        
    if savepath:
        data.to_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/{}_set.csv'.format(type_))
    return data

In [9]:
train_set = Remove_excess_negative_samples(train_set, train_set, 'train', )
independent_set = Remove_excess_negative_samples(independent_set, train_set, 'independent')
external_set = Remove_excess_negative_samples(external_set, train_set, 'external')

100%|██████████| 112/112 [14:20<00:00,  7.69s/it]


Remove num =  1800
Original:  Counter({0: 1797630, 1: 359166})
After remove:  Counter({0: 1795830, 1: 359166})


100%|██████████| 112/112 [00:15<00:00,  7.23it/s]


Remove num =  360
Original:  Counter({0: 86470, 1: 86110})
After remove:  Counter({1: 86110, 0: 86110})
After remove duplicates with train_set:  Counter({1: 85876, 0: 85562})


100%|██████████| 5/5 [00:00<00:00, 10.40it/s]


Remove num =  35
Original:  Counter({0: 52122, 1: 52087})
After remove:  Counter({1: 52087, 0: 52087})
After remove duplicates with train_set:  Counter({1: 51984, 0: 51881})


# 去掉independent和external在训练集中出现过的样本

In [10]:
def train_set_cv_fold_index(train_set):
    train_set_cv_idx_dict, test_set_cv_idx_dict = dict(), dict()   
    for hla in tqdm(set(train_set.HLA)):
        for length in range(8,15):

            temp_hla_length = train_set[train_set.HLA == hla][train_set.length == length]
            if len(temp_hla_length) != 0:
                temp_pos_hla_length_idx = temp_hla_length[temp_hla_length.label == 1].index

                test_num_hla_length = round(len(temp_pos_hla_length_idx) / 5)
                train_num_hla_length = len(temp_pos_hla_length_idx) - test_num_hla_length
                print('Positive {} - Length{}: # All = {} | # Train = {} | # Test = {}'.format(hla, length, len(temp_pos_hla_length_idx), train_num_hla_length, test_num_hla_length))

                pos_fold = []
                for i in range(5):       
                    if i != 4:
                        pos_fold.append(temp_pos_hla_length_idx[i * test_num_hla_length : (i+1) * test_num_hla_length])
                    else:
                        pos_fold.append(temp_pos_hla_length_idx[i * test_num_hla_length : ])

                train_fold, test_fold = [], []
                for fold in [0.0, 1.0, 2.0, 3.0, 4.0]:
                    i = int(fold)
                    temp_neg_hla_length_idx = temp_hla_length[temp_hla_length.label == 0][temp_hla_length.neg_fold == fold].index

                    cv_train_fold = [0, 1, 2, 3, 4]
                    cv_train_fold.remove(i)

                    pos_test_per_fold = pos_fold[i]
                    neg_test_per_fold = temp_neg_hla_length_idx[:len(pos_test_per_fold)]
                    pos_train_per_fold = []
                    for f in cv_train_fold:
                        pos_train_per_fold.extend(list(pos_fold[f]))
                    neg_train_per_fold = temp_neg_hla_length_idx[len(pos_test_per_fold) : len(pos_test_per_fold)+len(pos_train_per_fold)]

                    train_set_cv_idx_dict.setdefault(i, [])
                    test_set_cv_idx_dict.setdefault(i, [])
                    train_set_cv_idx_dict[i].extend(pos_train_per_fold)
                    train_set_cv_idx_dict[i].extend(neg_train_per_fold)
                    test_set_cv_idx_dict[i].extend(pos_test_per_fold)
                    test_set_cv_idx_dict[i].extend(neg_test_per_fold)
                    
    for fold in range(5):
        print(Counter(train_set.iloc[train_set_cv_idx_dict[fold]].label), Counter(train_set.iloc[test_set_cv_idx_dict[fold]].label))

    return train_set_cv_idx_dict, test_set_cv_idx_dict

In [11]:
def save_trainset_cvdata(train_set, train_data_cv_idx_dict, val_data_cv_idx_dict, fold, savepath = True):
    train_fold = train_set.loc[train_data_cv_idx_dict[fold]].sample(frac = 1.0).reset_index(drop = True)
    train_fold.drop(columns = ['neg_fold'], inplace = True)
    if savepath:
        train_fold.to_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/train_data_fold{}.csv'.format(fold))
        
    val_fold = train_set.loc[val_data_cv_idx_dict[0]].sample(frac = 1.0).reset_index(drop = True)
    val_fold.drop(columns = ['neg_fold'], inplace = True)
    if savepath:
        val_fold.to_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold{}.csv'.format(fold))

In [12]:
train_data_cv_idx_dict, val_data_cv_idx_dict = train_set_cv_fold_index(train_set)

  0%|          | 0/112 [00:00<?, ?it/s]

Positive HLA-B*27:01 - Length9: # All = 1512 | # Train = 1210 | # Test = 302
Positive HLA-B*27:01 - Length10: # All = 751 | # Train = 601 | # Test = 150
Positive HLA-B*27:01 - Length11: # All = 440 | # Train = 352 | # Test = 88
Positive HLA-B*27:01 - Length12: # All = 151 | # Train = 121 | # Test = 30
Positive HLA-B*27:01 - Length13: # All = 71 | # Train = 57 | # Test = 14


  1%|          | 1/112 [00:01<02:52,  1.56s/it]

Positive HLA-A*24:06 - Length9: # All = 92 | # Train = 74 | # Test = 18
Positive HLA-A*24:06 - Length10: # All = 76 | # Train = 61 | # Test = 15
Positive HLA-A*24:06 - Length11: # All = 74 | # Train = 59 | # Test = 15


  2%|▏         | 2/112 [00:03<02:50,  1.55s/it]

...

 99%|█████████▉| 111/112 [02:52<00:01,  1.54s/it]

Positive HLA-C*14:02 - Length8: # All = 388 | # Train = 310 | # Test = 78
Positive HLA-C*14:02 - Length9: # All = 1462 | # Train = 1170 | # Test = 292
Positive HLA-C*14:02 - Length10: # All = 253 | # Train = 202 | # Test = 51


100%|██████████| 112/112 [02:53<00:00,  1.55s/it]


Counter({1: 287329, 0: 287329}) Counter({1: 71837, 0: 71837})
Counter({1: 287329, 0: 287329}) Counter({1: 71837, 0: 71837})
Counter({1: 287329, 0: 287329}) Counter({1: 71837, 0: 71837})
Counter({1: 287329, 0: 287329}) Counter({1: 71837, 0: 71837})
Counter({1: 287348, 0: 287348}) Counter({1: 71818, 0: 71818})


NameError: name 'fold' is not defined

In [13]:
for fold in range(5):
    save_trainset_cvdata(train_set, train_data_cv_idx_dict, val_data_cv_idx_dict, fold, savepath = True)

In [14]:
Counter(pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold0.csv', index_col = 0).label)

Counter({0: 71837, 1: 71837})